In [4]:
#!pip install torchxrayvision matplotlib numpy pandas ipywidgets scikit-learn scikit-image seaborn

[TorchXRayVision: A library of chest X-ray datasets and models](https://arxiv.org/pdf/2111.00595)

https://github.com/naitik2314/Chest-X-Ray-Medical-Diagnosis-with-Deep-Learning

## Imports

In [5]:
# General utilities
import os
import sys
import platform
import time
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import urllib.request
import tarfile
from concurrent.futures import ThreadPoolExecutor

# PyTorch and data handling
import torch, torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, Subset

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# Image processing
# import skimage
# import skimage.transform

# Machine learning metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    balanced_accuracy_score,
    precision_recall_curve,
    auc,
)

# TorchXRayVision library
# import torchxrayvision as xrv

In [6]:
num_workers = max(1, os.cpu_count() // 2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# use_amp = True
# scaler = torch.amp.GradScaler(enabled=use_amp)

In [7]:
batch_size = 128
num_samples = 1000
num_epochs = 5
unique_patients = True
pathology_masks = True

In [8]:
machine_name = platform.node()  
user = os.getenv("USER") or os.getenv("USERNAME") 
os_name = platform.system()  # Get os
print(f"Machine: {machine_name}, User: {user}, OS: {os_name}")

if machine_name == "Corsair" and os_name == "Linux" and user == "jon":
    windows_drive = Path("/mnt/b/Xray")
    paths = {
        "dataset": windows_drive / "dataset",
        "tar_images": windows_drive / "dataset/images",
        "images": windows_drive / "dataset/images/images",
        "checkpoints": windows_drive / "checkpoints",
        "papers": windows_drive / "papers",
        "models": windows_drive / "models",
    }

    batch_size = 64

else:
    dataset_dir = Path("dataset")
    paths = {
        "dataset": dataset_dir,
        "tar_images": dataset_dir / "images",
        "images": dataset_dir / "images/images",
        "checkpoints": dataset_dir / "checkpoints",
        "papers": dataset_dir / "papers",
        "models": dataset_dir / "models",
    }

for key, path in paths.items():
    path.mkdir(parents=True, exist_ok=True)


Machine: DESKTOP-UHDJ875, User: jonal, OS: Linux


## Paths

In [9]:
dir_tar_images = paths['tar_images']

path_dataset = paths['dataset']
path_images = paths['images']
path_csv_list = paths['dataset'] / "Data_Entry_2017_v2020.csv"
path_train_val_list = paths['dataset'] / "train_val_list.txt"
path_test_list = paths['dataset'] / "test_list.txt"
path_models = paths['models']
checkpoint_dir = paths['checkpoints']

## Classes

## Functions

In [10]:
def download_file(link, folder, idx):
    """Downloads a file from a link to the specified folder."""
    file_name = f'images_{idx+1:03d}.tar.gz'
    file_path = os.path.join(folder, file_name)
    if os.path.exists(file_path):
        #print(f"{file_name} already exists, skipping download.")
        return file_path
    try:
        print(f"Downloading {file_name}...")
        urllib.request.urlretrieve(link, file_path)
        print(f"{file_name} downloaded successfully.")
        return file_path
    except Exception as e:
        print(f"Failed to download {file_name}: {e}")
        return None

In [11]:
def extract_file(file_path, folder):
    """Extracts a .tar.gz file to the specified folder."""
    extracted_flag = file_path.replace('.tar.gz', '_extracted.flag')
    if os.path.exists(extracted_flag):
        #print(f"{os.path.basename(file_path)} already extracted, skipping.")
        return
    try:
        print(f"Extracting {os.path.basename(file_path)}...")
        with tarfile.open(file_path, 'r:gz') as tar:
            tar.extractall(path=folder)
        with open(extracted_flag, 'w') as f:
            f.write('extracted')
        print(f"{os.path.basename(file_path)} extracted successfully.")
    except Exception as e:
        print(f"Failed to extract {os.path.basename(file_path)}: {e}")

In [12]:
def process_link(idx, link):
    """Handles downloading and extracting a single link."""
    file_path = download_file(link, dir_tar_images, idx)
    if file_path:
        extract_file(file_path, dir_tar_images)

In [13]:
def load_file_lists(train_val_path, test_path):
    """
    Load file lists from the provided paths.
    """
    with open(train_val_path, 'r') as f:
        train_val_files = [line.strip() for line in f.readlines()]
        
    with open(test_path, 'r') as f:
        test_files = [line.strip() for line in f.readlines()]
    
    return train_val_files, test_files

In [14]:
def split_dataset_by_file_list(dataset, train_val_files, test_files):
    """
    Split the dataset into train/validation and test sets based on file lists.
    
    Args:
        dataset: The full NIH_Dataset object.
        train_val_files: List of filenames for train/validation.
        test_files: List of filenames for test.
    
    Returns:
        train_val_dataset, test_dataset: Subsets of the original dataset.
    """
    # Create a mask for matching filenames
    train_val_mask = dataset.csv['Image Index'].isin(train_val_files)
    test_mask = dataset.csv['Image Index'].isin(test_files)

    # Get indices of matched files
    train_val_indices = dataset.csv[train_val_mask].index.tolist()
    test_indices = dataset.csv[test_mask].index.tolist()

    # Create dataset subsets
    train_val_dataset = Subset(dataset, train_val_indices)
    test_dataset = Subset(dataset, test_indices)

    return train_val_dataset, test_dataset

In [15]:
def load_checkpoint(checkpoint_path, model, optimizer=None, scaler=None):
    """
    Loads a checkpoint and restores the model, optimizer, and scaler states.

    Args:
        checkpoint_path: Path to the checkpoint file.
        model: The model to restore.
        optimizer: The optimizer to restore (optional).
        scaler: The gradient scaler to restore (optional).

    Returns:
        start_epoch: The epoch to continue training from.
        checkpoint: The full checkpoint dictionary.
    """
    if os.path.exists(checkpoint_path):
        print(f"Loading checkpoint from {checkpoint_path}...")
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])

        if optimizer:
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        if scaler:
            scaler.load_state_dict(checkpoint['scaler_state_dict'])

        start_epoch = checkpoint['epoch']
        print(f"Checkpoint loaded. Resuming from epoch {start_epoch}")
        return start_epoch, checkpoint
    else:
        print(f"No checkpoint found at {checkpoint_path}. Starting from scratch.")
        return 0, None


In [16]:
def plot_samples_with_masks(samples, dataset, max_samples=4):
    """
    Plot multiple X-ray images side-by-side along with their semantic and pathology masks (if available).
    Only plots samples with available masks, up to `max_samples`.
    """
    count = 0
    for sample in samples:
        if "pathology_masks" in sample and sample["pathology_masks"]:
            width = len(sample["pathology_masks"])
            fig, axs = plt.subplots(1, max(2, 1 + width), sharey=True, figsize=(3 + 3 * width, 3))
            axs[0].imshow(sample["img"][0], cmap="Greys_r")
            axs[0].set_title(f"Index: {sample['idx']}")
            for i, patho in enumerate(sample["pathology_masks"].keys()):
                axs[i + 1].imshow(sample["img"][0], cmap="Greys_r")
                axs[i + 1].imshow(sample["pathology_masks"][patho][0] + 1, alpha=0.5)
                axs[i + 1].set_title(dataset.pathologies[patho])
            plt.show()
            count += 1

        elif "semantic_masks" in sample and sample["semantic_masks"]:
            width = len(sample["semantic_masks"])
            fig, axs = plt.subplots(1, max(2, 1 + width), sharey=True, figsize=(3 + 3 * width, 3))
            axs[0].imshow(sample["img"][0], cmap="Greys_r")
            axs[0].set_title(f"Index: {sample['idx']}")
            for i, patho in enumerate(sample["semantic_masks"].keys()):
                axs[i + 1].imshow(sample["img"][0], cmap="Greys_r")
                axs[i + 1].imshow(sample["semantic_masks"][patho][0] + 1, alpha=0.5)
                axs[i + 1].set_title(patho)
            plt.show()
            count += 1

        if count >= max_samples:  # Stop after plotting max_samples
            break

    if count == 0:
        print("No samples with masks found.")


## Download data

In [17]:
links = [
    'https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz',
    'https://nihcc.box.com/shared/static/i28rlmbvmfjbl8p2n3ril0pptcmcu9d1.gz',
    'https://nihcc.box.com/shared/static/f1t00wrtdk94satdfb9olcolqx20z2jp.gz',
	'https://nihcc.box.com/shared/static/0aowwzs5lhjrceb3qp67ahp0rd1l1etg.gz',
    'https://nihcc.box.com/shared/static/v5e3goj22zr6h8tzualxfsqlqaygfbsn.gz',
	'https://nihcc.box.com/shared/static/asi7ikud9jwnkrnkj99jnpfkjdes7l6l.gz',
	'https://nihcc.box.com/shared/static/jn1b4mw4n6lnh74ovmcjb8y48h8xj07n.gz',
    'https://nihcc.box.com/shared/static/tvpxmn7qyrgl0w8wfh9kqfjskv6nmm1j.gz',
	'https://nihcc.box.com/shared/static/upyy3ml7qdumlgk2rfcvlb9k6gvqq2pj.gz',
	'https://nihcc.box.com/shared/static/l6nilvfa9cg3s28tqv1qc1olm3gnz54p.gz',
	'https://nihcc.box.com/shared/static/hhq8fkdgvcari67vfhs7ppg2w6ni4jze.gz',
	'https://nihcc.box.com/shared/static/ioqwiy20ihqwyr8pf4c24eazhh281pbu.gz'
]

with ThreadPoolExecutor(max_workers=num_workers) as executor:
    executor.map(lambda args: process_link(*args), enumerate(links))

print("Download and extraction complete. Please check the extracted files.")

images_001.tar.gz downloaded successfully.
Extracting images_001.tar.gz...
images_004.tar.gz downloaded successfully.
Extracting images_004.tar.gz...
images_008.tar.gz downloaded successfully.
Extracting images_008.tar.gz...
images_007.tar.gz downloaded successfully.
Extracting images_007.tar.gz...
images_002.tar.gz downloaded successfully.
Extracting images_002.tar.gz...
images_006.tar.gz downloaded successfully.
Extracting images_006.tar.gz...
images_005.tar.gz downloaded successfully.
Extracting images_005.tar.gz...
images_003.tar.gz downloaded successfully.
Extracting images_003.tar.gz...
images_001.tar.gz extracted successfully.
images_009.tar.gz downloaded successfully.
Extracting images_009.tar.gz...
images_004.tar.gz extracted successfully.
images_002.tar.gz extracted successfully.
images_008.tar.gz extracted successfully.
images_007.tar.gz extracted successfully.
images_005.tar.gz extracted successfully.
images_006.tar.gz extracted successfully.
images_003.tar.gz extracted suc

## Load data

In [14]:
transforms = transforms.Compose([
    xrv.datasets.XRayCenterCrop(),   # Crop the center of the image
    xrv.datasets.XRayResizer(224),  # Resize to 224x224
    # transforms.ToTensor(),          # Convert to tensor
    # transforms.Lambda(lambda x: x.unsqueeze(0) if x.dim() == 2 else x),  # Add channel dimension if missing
    # transforms.Lambda(lambda x: (x / 2048.0) * 1024.0)  # Normalize to [-1024, 1024]
])

In [15]:
dataset = xrv.datasets.NIH_Dataset(
    imgpath=path_images,
    csvpath=f"{path_dataset}/Data_Entry_2017_v2020.csv",
    transform=transforms,
    views=["PA", "AP"],
    unique_patients=unique_patients, # One image per patient
    pathology_masks=pathology_masks, # Load pathology masks
)
xrv.datasets.relabel_dataset(xrv.datasets.default_pathologies, dataset)

Lung Lesion doesn't exist. Adding nans instead.
Fracture doesn't exist. Adding nans instead.
Lung Opacity doesn't exist. Adding nans instead.
Enlarged Cardiomediastinum doesn't exist. Adding nans instead.


In [16]:
print(dataset)

{'Atelectasis': {0.0: 29140, 1.0: 1665},
 'Cardiomegaly': {0.0: 30028, 1.0: 777},
 'Consolidation': {0.0: 30369, 1.0: 436},
 'Edema': {0.0: 30727, 1.0: 78},
 'Effusion': {0.0: 29565, 1.0: 1240},
 'Emphysema': {0.0: 30539, 1.0: 266},
 'Enlarged Cardiomediastinum': {},
 'Fibrosis': {0.0: 30239, 1.0: 566},
 'Fracture': {},
 'Hernia': {0.0: 30724, 1.0: 81},
 'Infiltration': {0.0: 27354, 1.0: 3451},
 'Lung Lesion': {},
 'Lung Opacity': {},
 'Mass': {0.0: 29521, 1.0: 1284},
 'Nodule': {0.0: 29135, 1.0: 1670},
 'Pleural_Thickening': {0.0: 30051, 1.0: 754},
 'Pneumonia': {0.0: 30631, 1.0: 174},
 'Pneumothorax': {0.0: 30553, 1.0: 252}}
NIH_Dataset num_samples=30805 views=['PA', 'AP'] data_aug=None


In [17]:
train_val_files, test_files = load_file_lists(path_train_val_list, path_test_list)
train_val_dataset, test_dataset = split_dataset_by_file_list(dataset, train_val_files, test_files)

print(f"Train/Val Dataset: {len(train_val_dataset)} samples")
print(f"Test Dataset: {len(test_dataset)} samples")

Train/Val Dataset: 28008 samples
Test Dataset: 2797 samples


In [18]:
indices = list(range(num_samples))

train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)

train_dataset = Subset(train_val_dataset, train_indices)
val_dataset = Subset(train_val_dataset, val_indices)

print(f"Train Dataset: {len(train_dataset)} samples")
print(f"Validation Dataset: {len(val_dataset)} samples")

Train Dataset: 800 samples
Validation Dataset: 200 samples


In [19]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    prefetch_factor=num_workers
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
    prefetch_factor=num_workers
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
    prefetch_factor=num_workers
)

In [20]:
# for batch in train_loader:
#     images, labels = batch['img'], batch['lab']
#     print(f"Train batch: {images.shape}, {labels.shape}")
#     break

# for batch in val_loader:
#     images, labels = batch['img'], batch['lab']
#     print(f"Validation batch: {images.shape}, {labels.shape}")
#     break

# for batch in test_loader:
#     images, labels = batch['img'], batch['lab']
#     print(f"Test batch: {images.shape}, {labels.shape}")
#     break

## Load model

In [21]:
model = xrv.models.DenseNet(weights="nih").to(device)
model.op_threshs = None # Disable calibrated thresholds for the model

# dict(zip(model.pathologies,xrv.datasets.default_pathologies))

If this fails you can run `wget https://github.com/mlmed/torchxrayvision/releases/download/v1/nih-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best.pt -O C:\Users\jonin\.torchxrayvision\models_data/nih-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best.pt`
[██████████████████████████████████████████████████]


In [22]:
# Align dataset pathologies to model pathologies
common_pathologies = list(set(dataset.pathologies) & set(model.pathologies))
num_common_pathologies = len(common_pathologies)
print(f"Common Pathologies: {common_pathologies}")

# Map dataset indices to model indices
dataset_to_model_indices = {dataset.pathologies.index(p): model.pathologies.index(p) for p in common_pathologies}
print(f"Dataset to Model Index Mapping: {dataset_to_model_indices}")

Common Pathologies: ['Atelectasis', 'Effusion', 'Infiltration', 'Consolidation', 'Nodule', 'Edema', 'Pneumonia', 'Fibrosis', 'Cardiomegaly', 'Emphysema', 'Mass', 'Pleural_Thickening', 'Hernia', 'Pneumothorax']
Dataset to Model Index Mapping: {0: 0, 7: 7, 2: 2, 1: 1, 11: 11, 4: 4, 8: 8, 6: 6, 10: 10, 5: 5, 12: 12, 9: 9, 13: 13, 3: 3}


In [23]:
# Determine the number of output features dynamically
dummy_input = torch.zeros(1, 1, 224, 224)  # Batch size 1, single channel, 224x224 input
if torch.cuda.is_available():
    dummy_input = dummy_input.cuda()

# Get the output shape of the feature extractor
with torch.no_grad():
    num_features = model.features(dummy_input).shape[1]  # The second dimension is the feature size

# Update the classifier to match the number of pathologies
model.classifier = torch.nn.Linear(num_features, num_common_pathologies).to(device)
print(f"Updated classifier to output {num_common_pathologies} pathologies.")


Updated classifier to output 14 pathologies.


In [24]:
# Update classifier to match the number of common pathologies
model.classifier = torch.nn.Linear(num_features, num_common_pathologies).to(device)
print(f"Updated classifier to output {num_common_pathologies} pathologies.")

Updated classifier to output 14 pathologies.


## Optimizers

In [25]:
optimizer = torch.optim.Adam(model.classifier.parameters()) # only train classifier

## Loss

In [26]:
criterion = torch.nn.BCEWithLogitsLoss().to(device)

## Training

In [27]:
model_name = "first_model"
checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_{model_name}.pth")

# Load checkpoint
if os.path.exists(checkpoint_path):
    start_epoch, checkpoint = load_checkpoint(checkpoint_path, model, optimizer, scaler)
    if checkpoint:
        train_losses = checkpoint.get('train_losses', [])
        val_losses = checkpoint.get('val_losses', [])
        all_results = checkpoint.get('all_results', [])
else:
    start_epoch = 0
    train_losses = []
    val_losses = []
    all_results = []

for epoch in range(start_epoch, num_epochs):
    # ========================
    # Training
    # ========================
    model.train()
    epoch_train_losses = []
    for i, batch in enumerate(train_loader):
        # Prepare data
        batch_tensors = {k: v.to(device, non_blocking=True) for k, v in batch.items() if isinstance(v, torch.Tensor)}
        optimizer.zero_grad()

        # Extract targets and align for the model
        dataset_indices = list(dataset_to_model_indices.keys())
        model_indices = list(dataset_to_model_indices.values())
        targets = batch_tensors["lab"][:, dataset_indices].float().to(device)
        targets_aligned = torch.zeros((targets.size(0), len(model.pathologies)), device=device, dtype=torch.float)
        targets_aligned[:, model_indices] = targets

        # Forward pass with mixed precision
        with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
            img_input = batch_tensors["img"].to(torch.float16)  # Convert to HalfTensor
            outputs = model(img_input)
            loss = criterion(outputs[:, model_indices], targets_aligned[:, model_indices])

        # Backward pass
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        scaler.step(optimizer)
        scaler.update()

        # Log training loss
        epoch_train_losses.append(loss.item())

    # Store average training loss for this epoch
    avg_train_loss = np.mean(epoch_train_losses)
    train_losses.append(avg_train_loss)

    # ========================
    # Validation
    # ========================
    model.eval()
    outs, labs = [], []
    results = []  # Initialize for each epoch
    epoch_val_losses = []  # To store validation losses for the epoch
    with torch.no_grad():
        for batch in val_loader:
            # Prepare validation data
            batch_tensors = {k: v.to(device, non_blocking=True) for k, v in batch.items() if isinstance(v, torch.Tensor)}
            targets = batch_tensors["lab"][:, dataset_indices].float().to(device)
            targets_aligned = torch.zeros((targets.size(0), len(model.pathologies)), device=device, dtype=torch.float)
            targets_aligned[:, model_indices] = targets

            # Forward pass
            with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
                img_input = batch_tensors["img"]
                outputs = model(img_input)
                loss = criterion(outputs[:, model_indices], targets_aligned[:, model_indices])

            # Log validation loss for the batch
            epoch_val_losses.append(loss.item())

            # Log outputs and labels
            outs.extend(outputs[:, model_indices].detach().cpu().numpy())
            labs.extend(targets_aligned[:, model_indices].detach().cpu().numpy())


        # Calculate Metrics
        outs = np.array(outs)
        labs = np.array(labs)
        for idx, pathology in enumerate(common_pathologies):
            result = {"Pathology": pathology}
            if len(np.unique(labs[:, idx])) > 1:  # Only calculate metrics if labels are not constant
                labels = labs[:, idx].astype(bool)
                preds = outs[:, idx]
                result["AUC"] = roc_auc_score(labels, preds)
                result["Acc"] = accuracy_score(labels, preds > 0.5)
                result["F1"] = f1_score(labels, preds > 0.5)
                result["Precision"] = precision_score(labels, preds > 0.5)
                result["Recall"] = recall_score(labels, preds > 0.5)
                tn, fp, fn, tp = confusion_matrix(labels, preds > 0.5).ravel()
                result["Specificity"] = tn / (tn + fp)
                result["Balanced Accuracy"] = balanced_accuracy_score(labels, preds > 0.5)
                precision, recall, _ = precision_recall_curve(labels, preds)
                result["PR AUC"] = auc(recall, precision)
            results.append(result)
        
        # Append to all_results
        if len(results) > 0:
            df_results = pd.DataFrame(results)
            all_results.append(df_results)
            #print(f"Epoch {epoch + 1} Evaluation Results:")
            #display(df_results)
        else:
            print(f"No metrics calculated for epoch {epoch + 1}.")
    
    # Store average validation loss for the epoch
    avg_val_loss = np.mean(epoch_val_losses)
    val_losses.append(avg_val_loss)

    print(f"Epoch {epoch + 1} - Training Loss: {avg_train_loss:.4f}, Validation Loss: {loss.item():.4f}")

    # ========================
    # Log and Save Checkpoints
    # ========================
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler_state_dict': scaler.state_dict(),
        'train_losses': train_losses,  # Save loss history
        'val_losses': val_losses,
        'all_results': all_results,   # Save all results
    }, checkpoint_path)

# ========================
# Loss Curve
# ========================
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss', marker='o')
plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss', marker='s')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Curve')
plt.legend()
plt.grid()
plt.show()

# ========================
# Final Results
# ========================
if len(all_results) > 0:
    final_results = pd.concat(all_results, keys=range(1, len(all_results) + 1))
    final_results.to_csv(os.path.join(checkpoint_dir, "evaluation_results.csv"), index=False)
    print("Evaluation results saved.")

    # Summary of metrics (averages across epochs)
    metrics_summary = final_results.groupby("Pathology").mean()
    print("\nSummary of Metrics Across Epochs:")
    display(metrics_summary)

    # Save the summary
    metrics_summary.to_csv(os.path.join(checkpoint_dir, "metrics_summary.csv"), index=True)
else:
    print("No evaluation results to save.")

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\jonin\.conda\envs\pytorch_24\Lib\site-packages\torch\utils\data\_utils\worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jonin\.conda\envs\pytorch_24\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 50, in fetch
    data = self.dataset.__getitems__(possibly_batched_index)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jonin\.conda\envs\pytorch_24\Lib\site-packages\torch\utils\data\dataset.py", line 418, in __getitems__
    return self.dataset.__getitems__([self.indices[idx] for idx in indices])  # type: ignore[attr-defined]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jonin\.conda\envs\pytorch_24\Lib\site-packages\torch\utils\data\dataset.py", line 420, in __ge